# Aggregating and Combining `pandas` DataFrames

In [1]:
import pandas as pd
import numpy as np
import requests as rq
from sklearn.preprocessing import OneHotEncoder
from zipfile import ZipFile

## Learning Goals

- Use GroupBy objects to organize and aggregate data
- Create pivot tables from DataFrames
- Combine DataFrames by merging and appending 

We'll work with the Austin Animal Center dataset and with data from King County's Department of Assessments (Seattle housing data).

### Austin Animal Center Data

In [2]:
data = rq.get('https://data.austintexas.gov/resource/9t4d-g238.json').text

In [3]:
animals = pd.read_json(data)

In [4]:
animals.head()

,animal_id,datetime,monthyear,date_of_birth,outcome_type,outcome_subtype,animal_type,sex_upon_outcome,age_upon_outcome,breed,color,name
0,A853501,2022-03-23 13:59:00,2022-03-23T13:59:00.000,2020-03-19T00:00:00.000,Euthanasia,Rabies Risk,Other,Unknown,2 years,Bat,Brown,NaN
1,A853709,2022-03-23 10:12:00,2022-03-23T10:12:00.000,2020-03-23T00:00:00.000,Euthanasia,Rabies Risk,Other,Unknown,NULL,Bat,Brown,NaN
2,A852545,2022-03-23 10:03:00,2022-03-23T10:03:00.000,2022-01-12T00:00:00.000,Adoption,Foster,Dog,Spayed Female,2 months,Boxer Mix,Brown/White,*Pico
3,A853653,2022-03-23 08:35:00,2022-03-23T08:35:00.000,2021-03-22T00:00:00.000,Euthanasia,Rabies Risk,Other,Unknown,1 year,Bat,Brown,NaN
4,A853381,2022-03-22 19:01:00,2022-03-22T19:01:00.000,2020-03-17T00:00:00.000,Adoption,NaN,Dog,Spayed Female,2 years,Basenji Mix,Black/Tan,*Korinna


In [5]:
animals.dtypes

animal_id                   object
datetime            datetime64[ns]
monthyear                   object
date_of_birth               object
outcome_type                object
outcome_subtype             object
animal_type                 object
sex_upon_outcome            object
age_upon_outcome            object
breed                       object
color                       object
name                        object
dtype: object

## Aggregating over DataFrames: `.groupby()`

Those of you familiar with SQL have probably used the GROUP BY command. (And if you haven't, you'll see it very soon!) Pandas has this, too.

The `.groupby()` method is especially useful for aggregate functions applied to the data grouped in particular ways.

In [6]:
animals.groupby('animal_type')

In [7]:
animals.columns

Index(['animal_id', 'datetime', 'monthyear', 'date_of_birth', 'outcome_type',
       'outcome_subtype', 'animal_type', 'sex_upon_outcome',
       'age_upon_outcome', 'breed', 'color', 'name'],
      dtype='object')

We can group by multiple columns, and also return a DataFrameGroupBy object

Notice the object type [DataFrameGroupBy](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html) object. 

### `.groups` and `.get_group()`

In [8]:
animals.groupby(['animal_type', 'outcome_type'])

In [9]:
animals.head()

,animal_id,datetime,monthyear,date_of_birth,outcome_type,outcome_subtype,animal_type,sex_upon_outcome,age_upon_outcome,breed,color,name
0,A853501,2022-03-23 13:59:00,2022-03-23T13:59:00.000,2020-03-19T00:00:00.000,Euthanasia,Rabies Risk,Other,Unknown,2 years,Bat,Brown,NaN
1,A853709,2022-03-23 10:12:00,2022-03-23T10:12:00.000,2020-03-23T00:00:00.000,Euthanasia,Rabies Risk,Other,Unknown,NULL,Bat,Brown,NaN
2,A852545,2022-03-23 10:03:00,2022-03-23T10:03:00.000,2022-01-12T00:00:00.000,Adoption,Foster,Dog,Spayed Female,2 months,Boxer Mix,Brown/White,*Pico
3,A853653,2022-03-23 08:35:00,2022-03-23T08:35:00.000,2021-03-22T00:00:00.000,Euthanasia,Rabies Risk,Other,Unknown,1 year,Bat,Brown,NaN
4,A853381,2022-03-22 19:01:00,2022-03-22T19:01:00.000,2020-03-17T00:00:00.000,Adoption,NaN,Dog,Spayed Female,2 years,Basenji Mix,Black/Tan,*Korinna


In [10]:
# This retuns each group indexed by the group name: I.E. 'Bird', along with the row indices of each value
# .group tells me which row belongs to which animal

animals.groupby('animal_type').groups

{'Bird': [457, 543, 544, 545, 546, 547, 548, 549, 550, 551, 552, 553, 554, 555, 556, 557, 558], 'Cat': [5, 6, 8, 11, 12, 15, 16, 18, 19, 23, 26, 29, 33, 35, 38, 43, 44, 47, 48, 49, 50, 51, 54, 70, 72, 73, 77, 78, 87, 89, 97, 99, 103, 109, 112, 113, 119, 127, 130, 135, 137, 138, 141, 148, 151, 152, 153, 154, 156, 170, 171, 177, 182, 183, 185, 188, 191, 194, 200, 202, 207, 208, 209, 214, 217, 218, 221, 245, 246, 247, 248, 249, 257, 261, 263, 276, 278, 279, 280, 281, 285, 286, 293, 298, 299, 302, 307, 328, 331, 334, 335, 337, 354, 364, 366, 368, 369, 370, 382, 384, ...], 'Dog': [2, 4, 7, 9, 10, 13, 14, 17, 20, 21, 22, 24, 25, 27, 28, 30, 31, 32, 34, 36, 39, 40, 41, 42, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 71, 74, 75, 76, 79, 80, 81, 82, 83, 84, 85, 86, 88, 90, 91, 92, 93, 94, 95, 96, 98, 100, 101, 102, 104, 105, 107, 108, 116, 117, 118, 120, 121, 122, 123, 124, 126, 128, 131, 132, 133, 134, 136, 139, 140, 142, 143, 144, 145, 146, 147, 149, 150, 155, 158, 159, 160, 1

Once we know we are working with a type of object, it opens up a suite of attributes and methods. One attribute we can look at is groups.

In [11]:
# Once we know the group indices, we can return the groups using those indices.
animals.groupby('animal_type').get_group('Dog')

,animal_id,datetime,monthyear,date_of_birth,outcome_type,outcome_subtype,animal_type,sex_upon_outcome,age_upon_outcome,breed,color,name
2,A852545,2022-03-23 10:03:00,2022-03-23T10:03:00.000,2022-01-12T00:00:00.000,Adoption,Foster,Dog,Spayed Female,2 months,Boxer Mix,Brown/White,*Pico
4,A853381,2022-03-22 19:01:00,2022-03-22T19:01:00.000,2020-03-17T00:00:00.000,Adoption,NaN,Dog,Spayed Female,2 years,Basenji Mix,Black/Tan,*Korinna
7,A853426,2022-03-22 17:56:00,2022-03-22T17:56:00.000,2022-01-06T00:00:00.000,Adoption,NaN,Dog,Neutered Male,2 months,Chinese Sharpei Mix,Buff/Black,*Flycatcher
9,A853629,2022-03-22 16:43:00,2022-03-22T16:43:00.000,2021-07-22T00:00:00.000,Return to Owner,Field,Dog,Intact Male,7 months,Siberian Husky Mix,White,Oso
10,A852933,2022-03-22 16:32:00,2022-03-22T16:32:00.000,2021-08-10T00:00:00.000,Return to Owner,NaN,Dog,Spayed Female,7 months,Pit Bull,White/Gray,Indica
...,...,...,...,...,...,...,...,...,...,...,...,...
990,A851412,2022-02-13 13:16:00,2022-02-13T13:16:00.000,2021-02-11T00:00:00.000,Transfer,Partner,Dog,Intact Female,1 year,Pit Bull Mix,Fawn/White,Nellie
991,A851434,2022-02-13 13:15:00,2022-02-13T13:15:00.000,2010-02-12T00:00:00.000,Transfer,Partner,Dog,Neutered Male,12 years,Chihuahua Shorthair,Fawn/Tan,Taco Bell
992,A849153,2022-02-13 12:00:00,2022-02-13T12:00:00.000,2021-01-03T00:00:00.000,Transfer,Partner,Dog,Spayed Female,1 year,Rottweiler,Black/Brown,Natasha
996,A851303,2022-02-12 18:58:00,2022-02-12T18:58:00.000,2020-02-09T00:00:00.000,Transfer,Partner,Dog,Intact Female,2 years,Cairn Terrier Mix,Brown/White,A851303


#### Multi-Indexing

In [12]:
# Same goes for multi index groupbys
animal_outcome = animals.groupby(['animal_type', 'outcome_type'])
animal_outcome.groups

{('Bird', 'Adoption'): [457, 543, 544, 545, 546, 547, 548, 549, 550, 551, 552, 553, 554, 555, 556, 557, 558], ('Cat', 'Adoption'): [5, 6, 8, 11, 12, 15, 16, 18, 19, 23, 26, 29, 33, 35, 38, 54, 70, 77, 78, 89, 97, 99, 112, 113, 119, 135, 137, 138, 141, 148, 154, 170, 171, 177, 185, 188, 191, 194, 200, 202, 207, 208, 209, 217, 218, 261, 263, 276, 278, 279, 280, 281, 285, 293, 298, 299, 302, 307, 328, 331, 334, 335, 337, 354, 364, 368, 369, 370, 392, 394, 395, 396, 398, 401, 402, 406, 407, 414, 421, 422, 424, 438, 444, 445, 453, 469, 470, 473, 486, 492, 499, 519, 573, 580, 593, 594, 601, 607, 620, 621, ...], ('Cat', 'Died'): [588, 631], ('Cat', 'Euthanasia'): [87, 103, 127, 130, 156, 366, 475, 476, 477, 560, 627, 721, 805, 834, 913, 974], ('Cat', 'Return to Owner'): [153, 286, 382, 461, 542, 600, 803, 804, 912, 924, 936, 987], ('Cat', 'Rto-Adopt'): [516], ('Cat', 'Transfer'): [43, 44, 47, 48, 49, 50, 51, 72, 73, 109, 151, 152, 182, 183, 214, 221, 245, 246, 247, 248, 249, 257, 384, 385, 38

In [13]:
# animal_outcome.groups is a dictionary, so we can access the group names using keys()
animal_outcome.groups.keys()

dict_keys([('Bird', 'Adoption'), ('Cat', 'Adoption'), ('Cat', 'Died'), ('Cat', 'Euthanasia'), ('Cat', 'Return to Owner'), ('Cat', 'Rto-Adopt'), ('Cat', 'Transfer'), ('Dog', 'Adoption'), ('Dog', 'Died'), ('Dog', 'Euthanasia'), ('Dog', 'Return to Owner'), ('Dog', 'Rto-Adopt'), ('Dog', 'Transfer'), ('Other', 'Adoption'), ('Other', 'Disposal'), ('Other', 'Euthanasia'), ('Other', 'Return to Owner'), ('Other', 'Transfer')])

In [14]:
animal_outcome.groups.values()

dict_values([Int64Index([457, 543, 544, 545, 546, 547, 548, 549, 550, 551, 552, 553, 554,
            555, 556, 557, 558],
           dtype='int64'), Int64Index([  5,   6,   8,  11,  12,  15,  16,  18,  19,  23,
            ...
            963, 964, 965, 969, 976, 977, 982, 983, 986, 999],
           dtype='int64', length=179), Int64Index([588, 631], dtype='int64'), Int64Index([87, 103, 127, 130, 156, 366, 475, 476, 477, 560, 627, 721, 805,
            834, 913, 974],
           dtype='int64'), Int64Index([153, 286, 382, 461, 542, 600, 803, 804, 912, 924, 936, 987], dtype='int64'), Int64Index([516], dtype='int64'), Int64Index([ 43,  44,  47,  48,  49,  50,  51,  72,  73, 109, 151, 152, 182,
            183, 214, 221, 245, 246, 247, 248, 249, 257, 384, 385, 386, 387,
            388, 389, 390, 430, 433, 435, 440, 520, 521, 522, 565, 583, 584,
            585, 586, 587, 623, 625, 626, 628, 739, 763, 806, 807, 808, 824,
            825, 826, 827, 843, 875, 891, 893, 894, 978, 981],
      

In [15]:
# We can then get a specific group, such as Cats that were adopted
animal_outcome.get_group(('Cat', 'Adoption'))

,animal_id,datetime,monthyear,date_of_birth,outcome_type,outcome_subtype,animal_type,sex_upon_outcome,age_upon_outcome,breed,color,name
5,A534157,2022-03-22 18:22:00,2022-03-22T18:22:00.000,2008-05-30T00:00:00.000,Adoption,NaN,Cat,Spayed Female,13 years,Domestic Shorthair Mix,Brown Tabby,Sasha Bell
6,A853445,2022-03-22 18:02:00,2022-03-22T18:02:00.000,2021-10-18T00:00:00.000,Adoption,NaN,Cat,Spayed Female,5 months,Domestic Shorthair,Brown Tabby,Petunia
8,A852038,2022-03-22 17:54:00,2022-03-22T17:54:00.000,2021-02-28T00:00:00.000,Adoption,NaN,Cat,Neutered Male,1 year,Domestic Shorthair,Black,*Minkus
11,A853444,2022-03-22 15:43:00,2022-03-22T15:43:00.000,2020-03-18T00:00:00.000,Adoption,NaN,Cat,Spayed Female,2 years,Domestic Shorthair,Brown Tabby,Hope
12,A849460,2022-03-22 15:10:00,2022-03-22T15:10:00.000,2021-01-08T00:00:00.000,Adoption,Foster,Cat,Neutered Male,1 year,Domestic Shorthair,Orange Tabby,*Gingerbread
...,...,...,...,...,...,...,...,...,...,...,...,...
977,A850396,2022-02-14 16:08:00,2022-02-14T16:08:00.000,2020-01-31T00:00:00.000,Adoption,NaN,Cat,Spayed Female,2 years,Domestic Shorthair,Brown Tabby,Lovebug
982,A851280,2022-02-14 15:06:00,2022-02-14T15:06:00.000,2020-06-09T00:00:00.000,Adoption,NaN,Cat,Spayed Female,1 year,Domestic Medium Hair,White/Black,Gemma
983,A850855,2022-02-14 14:39:00,2022-02-14T14:39:00.000,2021-10-23T00:00:00.000,Adoption,NaN,Cat,Neutered Male,3 months,Domestic Shorthair,Orange Tabby/White,*Clark
986,A846503,2022-02-14 12:33:00,2022-02-14T12:33:00.000,2021-09-08T00:00:00.000,Adoption,NaN,Cat,Spayed Female,5 months,Domestic Shorthair Mix,Tortie,*Pandora


### Aggregating

Once again, as we will see in SQL, groupby objects are intended to be used with aggregation. In SQL, we will see that our queries that include GROUP BY require aggregation performed on columns.

We can use `.sum()`, `.mean()`, `.count()`, `.max()`, `.min()`, etc. Find a list of common aggregations [here](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html).

In [16]:
animals.groupby('animal_type').count()

,animal_id,datetime,monthyear,date_of_birth,outcome_type,outcome_subtype,sex_upon_outcome,age_upon_outcome,breed,color,name
animal_type,,,,,,,,,,,
Bird,17,17,17,17,17,0,17,17,17,17,1
Cat,272,272,272,272,272,105,272,272,272,272,235
Dog,649,649,649,649,649,244,649,649,649,649,602
Other,62,62,62,62,62,55,62,62,62,62,20


### Exercise

Use `.groupby()` to find the most recently born of each (main) animal type.

In [17]:
animals.groupby('animal_type')['date_of_birth'].max()

animal_type
Bird     2020-03-01T00:00:00.000
Cat      2022-03-19T00:00:00.000
Dog      2022-03-13T00:00:00.000
Other    2021-07-02T00:00:00.000
Name: date_of_birth, dtype: object

<details>
    <summary>Answer</summary>
    <code>animals.groupby('animal_type')['date_of_birth'].max()</code>
    </details>

## Pivoting a DataFrame

### `.pivot_table()`

Those of you familiar with Excel have probably used Pivot Tables. Pandas has a similar functionality.

Grouping by two different columns can be very helpful.

In [18]:
animals.groupby(by=['outcome_type', 'sex_upon_outcome']).agg(len)

animal_id  datetime  monthyear  \
outcome_type    sex_upon_outcome                                   
Adoption        Intact Female            26        26         26   
                Intact Male               7         7          7   
                Neutered Male           254       254        254   
                Spayed Female           248       248        248   
Died            Intact Female             1         1          1   
                Intact Male               2         2          2   
                Neutered Male             2         2          2   
                Unknown                   1         1          1   
Disposal        Unknown                   1         1          1   
Euthanasia      Intact Female             6         6          6   
                Intact Male               9         9          9   
                Neutered Male             2         2          2   
                Spayed Female             1         1          1   
                Unknown                  51        51         51   
Return to Owner Intact Female            30        30         30   
                Intact Male              43        43         43   
                Neutered Male            41        41         41   
                Spayed Female            33        33         33   
                Unknown                   2         2          2   
Rto-Adopt       Intact Female             1         1          1   
                Neutered Male             8         8          8   
                Spayed Female             5         5          5   
Transfer        Intact Female            63        63         63   
                Intact Male              54        54         54   
                Neutered Male            57        57         57   
                Spayed Female            44        44         44   
                Unknown                   8         8          8   

                                  date_of_birth  outcome_subtype  animal_type  \
outcome_type    sex_upon_outcome                                                
Adoption        Intact Female                26               26           26   
                Intact Male                   7                7            7   
                Neutered Male               254              254          254   
                Spayed Female               248              248          248   
Died            Intact Female                 1                1            1   
                Intact Male                   2                2            2   
                Neutered Male                 2                2            2   
                Unknown                       1                1            1   
Disposal        Unknown                       1                1            1   
Euthanasia      Intact Female                 6                6            6   
                Intact Male                   9                9            9   
                Neutered Male                 2                2            2   
                Spayed Female                 1                1            1   
                Unknown                      51               51           51   
Return to Owner Intact Female                30               30           30   
                Intact Male                  43               43           43   
                Neutered Male                41               41           41   
                Spayed Female                33               33           33   
                Unknown                       2                2            2   
Rto-Adopt       Intact Female                 1                1            1   
                Neutered Male                 8                8            8   
                Spayed Female                 5                5            5   
Transfer        Intact Female                63               63           63   
                Intact Male         

But it has the unsavory side effect of creating a two-level index. This can be a good time to use `.pivot_table()`.

(There is also a `.pivot()`. For the somewhat subtle differences, see [here](https://stackoverflow.com/questions/30960338/pandas-difference-between-pivot-and-pivot-table-why-is-only-pivot-table-workin).)

#### Example

In [19]:
df = pd.DataFrame({"sex": ["male", "male", "male", "male", "male",
                          "female", "female", "female", "female"],
                    "num_puppies": ["one", "one", "one", "two", "two",
                          "one", "one", "two", "two"],
                    "breed": ["terrier", "retriever", "retriever", "terrier",
                          "terrier", "retriever", "terrier", "terrier",
                          "retriever"],
                    "past_owners": [1, 2, 2, 3, 3, 4, 5, 6, 7],
                    "family_members": [2, 4, 5, 5, 6, 6, 8, 9, 9]})
df

,sex,num_puppies,breed,past_owners,family_members
0,male,one,terrier,1,2
1,male,one,retriever,2,4
2,male,one,retriever,2,5
3,male,two,terrier,3,5
4,male,two,terrier,3,6
5,female,one,retriever,4,6
6,female,one,terrier,5,8
7,female,two,terrier,6,9
8,female,two,retriever,7,9


In [20]:
# This first example aggregates values by taking the sum.

table = pd.pivot_table(df, values='past_owners', index=['sex', 'num_puppies'],
                     columns=['breed'], aggfunc=np.sum)
table

breed               retriever  terrier
sex    num_puppies                    
female one                4.0      5.0
       two                7.0      6.0
male   one                4.0      1.0
       two                NaN      6.0

In [21]:
table.index

MultiIndex([('female', 'one'),
            ('female', 'two'),
            (  'male', 'one'),
            (  'male', 'two')],
           names=['sex', 'num_puppies'])

In [22]:
table.reset_index() # flatten

breed,sex,num_puppies,retriever,terrier
0,female,one,4.0,5.0
1,female,two,7.0,6.0
2,male,one,4.0,1.0
3,male,two,NaN,6.0


#### Back to Austin animals

In [23]:
animals.pivot_table(index='outcome_type', columns='sex_upon_outcome', aggfunc=len)

age_upon_outcome                                          \
sex_upon_outcome    Intact Female Intact Male Neutered Male Spayed Female   
outcome_type                                                                
Adoption                     26.0         7.0         254.0         248.0   
Died                          1.0         2.0           2.0           NaN   
Disposal                      NaN         NaN           NaN           NaN   
Euthanasia                    6.0         9.0           2.0           1.0   
Return to Owner              30.0        43.0          41.0          33.0   
Rto-Adopt                     1.0         NaN           8.0           5.0   
Transfer                     63.0        54.0          57.0          44.0   

                             animal_id                            \
sex_upon_outcome Unknown Intact Female Intact Male Neutered Male   
outcome_type                                                       
Adoption             NaN          26.0         7.0         254.0   
Died                 1.0           1.0         2.0           2.0   
Disposal             1.0           NaN         NaN           NaN   
Euthanasia          51.0           6.0         9.0           2.0   
Return to Owner      2.0          30.0        43.0          41.0   
Rto-Adopt            NaN           1.0         NaN           8.0   
Transfer             8.0          63.0        54.0          57.0   

                                        ...          name              \
sex_upon_outcome Spayed Female Unknown  ... Intact Female Intact Male   
outcome_type                            ...                             
Adoption                 248.0     NaN  ...          26.0         7.0   
Died                       NaN     1.0  ...           1.0         2.0   
Disposal                   NaN     1.0  ...           NaN         NaN   
Euthanasia                 1.0    51.0  ...           6.0         9.0   
Return to Owner           33.0     2.0  ...          30.0        43.0   
Rto-Adopt                  5.0     NaN  ...           1.0         NaN   
Transfer                  44.0     8.0  ...          63.0        54.0   

                                                     outcome_subtype  \
sex_upon_outcome Neutered Male Spayed Female Unknown   Intact Female   
outcome_type                                                           
Adoption                 254.0         248.0     NaN            26.0   
Died                       2.0           NaN     1.0             1.0   
Disposal                   NaN           NaN     1.0             NaN   
Euthanasia                 2.0           1.0    51.0             6.0   
Return to Owner           41.0          33.0     2.0            30.0   
Rto-Adopt                  8.0           5.0     NaN             1.0   
Transfer                  57.0          44.0     8.0            63.0   

                                                                  
sex_upon_outcome Intact Male Neutered Male Spayed Female Unknown  
outcome_type                                                      
Adoption                 7.0         254.0         248.0     NaN  
Died                     2.0           2.0           NaN     1.0  
Disposal                 NaN           NaN           NaN     1.0  
Euthanasia               9.0           2.0           1.0    51.0  
Return to Owner         43.0          41.0          33.0     2.0  
Rto-Adopt                NaN           8.0           5.0     NaN  
Transfer                54.0          57.0          44.0     8.0  

[7 rows x 50 columns]

### Exercise

Use `.pivot_table()` to add up the number of my tasks by category. Hint: Use `sum()` as your aggregating function.

In [24]:
tasks = pd.DataFrame({'category': ['house', 'house', 'school', 'school'],
                      'descr': ['kitchen', 'laundry', 'git', 'Python'],
                      'priority': [2, 3, 4, 1], 'num_tasks': [2, 1, 2, 3]})

tasks

,category,descr,priority,num_tasks
0,house,kitchen,2,2
1,house,laundry,3,1
2,school,git,4,2
3,school,Python,1,3


In [25]:
tasks.pivot_table(index='category', values='num_tasks', aggfunc=sum)

,num_tasks
category,
house,3
school,5


<details>
    <summary>Answer</summary>
    <code>tasks.pivot_table(values='num_tasks', index='category', aggfunc=sum)</code>
    </details>

## Methods for Combining DataFrames: `.join()`, `.merge()`, `pd.concat()`

### `.join()`

In [26]:
toy1 = pd.DataFrame([[63, 142], [33, 47]], columns=['age', 'HP'])
toy2 = pd.DataFrame([[63, 100], [33, 200]], columns=['age', 'MP'])

toy1

,age,HP
0,63,142
1,33,47


In [27]:
toy2

,age,MP
0,63,100
1,33,200


In [28]:
# We can't just join these as they are, since we haven't specified our suffixes.

toy1.join(toy2)

ValueError: columns overlap but no suffix specified: Index(['age'], dtype='object')

In [29]:
toy1.join(toy2, lsuffix='1', rsuffix='2')

,age1,HP,age2,MP
0,63,142,63,100
1,33,47,33,200


If we don't want to keep both, we could set the overlapping column as the index in each DataFrame:

In [30]:
toy1.set_index('age').join(toy2.set_index('age'))

,HP,MP
age,,
63,142,100
33,47,200


For more on this method, check out the [doc](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.join.html)!

### `.merge()`

Or we could use `.merge()`:

In [31]:
toy1.merge(toy2)

,age,HP,MP
0,63,142,100
1,33,47,200


In [32]:
ds_chars = pd.read_csv('data/ds_chars.csv', index_col=0)
ds_chars

,name,HP,home_state
0,greg,200,WA
1,miles,200,WA
2,alan,170,TX
3,alison,300,DC
4,rachel,200,TX


In [33]:
states = pd.read_csv('data/states.csv', index_col=0)
states

,state,nickname,capital
0,WA,evergreen,Olympia
1,TX,alamo,Austin
2,DC,district,Washington
3,OH,buckeye,Columbus
4,OR,beaver,Salem


### The `how` Parameter

This parameter in both `.join()` and `.merge()` tells the compiler what sort of join to effect. We'll cover this in detail when we discuss SQL.

In [34]:
ds_chars.merge(states,
               left_on='home_state',
               right_on='state',
               how='inner')

,name,HP,home_state,state,nickname,capital
0,greg,200,WA,WA,evergreen,Olympia
1,miles,200,WA,WA,evergreen,Olympia
2,alan,170,TX,TX,alamo,Austin
3,rachel,200,TX,TX,alamo,Austin
4,alison,300,DC,DC,district,Washington


In [35]:
ds_chars.merge(states,
               left_on='home_state',
               right_on='state',
               how='outer')

,name,HP,home_state,state,nickname,capital
0,greg,200.0,WA,WA,evergreen,Olympia
1,miles,200.0,WA,WA,evergreen,Olympia
2,alan,170.0,TX,TX,alamo,Austin
3,rachel,200.0,TX,TX,alamo,Austin
4,alison,300.0,DC,DC,district,Washington
5,NaN,NaN,NaN,OH,buckeye,Columbus
6,NaN,NaN,NaN,OR,beaver,Salem


### `pd.concat()`

This method takes a *list* of pandas objects as arguments.

In [36]:
ds_full = pd.concat([ds_chars, states])
ds_full

,name,HP,home_state,state,nickname,capital
0,greg,200.0,WA,NaN,NaN,NaN
1,miles,200.0,WA,NaN,NaN,NaN
2,alan,170.0,TX,NaN,NaN,NaN
3,alison,300.0,DC,NaN,NaN,NaN
4,rachel,200.0,TX,NaN,NaN,NaN
0,NaN,NaN,NaN,WA,evergreen,Olympia
1,NaN,NaN,NaN,TX,alamo,Austin
2,NaN,NaN,NaN,DC,district,Washington
3,NaN,NaN,NaN,OH,buckeye,Columbus
4,NaN,NaN,NaN,OR,beaver,Salem


`pd.concat()`–– and many other pandas operations –– make use of an `axis` parameter. For this particular method I need to specify whether I want to concatenate the DataFrames *row-wise* (`axis=0`) or *column-wise* (`axis=1`). The default is `axis=0`, so let's override that!

In [37]:
ds_full = pd.concat([ds_chars, states], axis=1)
ds_full

,name,HP,home_state,state,nickname,capital
0,greg,200,WA,WA,evergreen,Olympia
1,miles,200,WA,TX,alamo,Austin
2,alan,170,TX,DC,district,Washington
3,alison,300,DC,OH,buckeye,Columbus
4,rachel,200,TX,OR,beaver,Salem


## King County Assessments

As data scientists, we want to build a model to predict the sale price of a house in Seattle in 2019, based on its square footage. We know that the King County Department of Assessments has comprehensive data available on real property sales in the Seattle area. We need to prepare the data.

### First, get the data!

Go [here](https://info.kingcounty.gov/assessor/DataDownload/default.aspx) and download two files: "Real Property Sales" and "Residential Building". Then unzip them. (Or you can run the cells below if you prefer.)

In [ ]:
# %%bash
# cd data
# curl -o property_sales.zip https://aqua.kingcounty.gov/extranet/assessor/Real%20Property%20Sales.zip 

In [ ]:
# %%bash
# cd data
# curl -o res_bldg.zip https://aqua.kingcounty.gov/extranet/assessor/Residential%20Building.zip 

In [ ]:
# zf = ZipFile('data/property_sales.zip', 'r')
# zf.extractall('data')
# zf.close()

In [ ]:
# zf = ZipFile('data/res_bldg.zip', 'r')
# zf.extractall('data')
# zf.close()

In [38]:
# You'll need to use a new encoding here. List of all encodings here:
# https://docs.python.org/3/library/codecs.html#standard-encodings

# Both of these csv files have many columns, so we'll just pre-select
# which ones we want to use.

sales_df = pd.read_csv('data/EXTR_RPSale.csv',
                       encoding='latin-1',
                       usecols=['Major', 'Minor', 'DocumentDate', 'SalePrice'])

/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (1,2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [39]:
sales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2220587 entries, 0 to 2220586
Data columns (total 4 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   Major         object
 1   Minor         object
 2   DocumentDate  object
 3   SalePrice     int64 
dtypes: int64(1), object(3)
memory usage: 67.8+ MB


In [40]:
bldg_df = pd.read_csv('data/EXTR_ResBldg.csv',
                     usecols=['Major', 'Minor', 'SqFtTotLiving', 'ZipCode'])

/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [41]:
bldg_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 522030 entries, 0 to 522029
Data columns (total 4 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   Major          522030 non-null  int64 
 1   Minor          522030 non-null  int64 
 2   ZipCode        470868 non-null  object
 3   SqFtTotLiving  522030 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 15.9+ MB


In [42]:
sales_data = pd.merge(sales_df, bldg_df, on=['Major', 'Minor'])

In [43]:
sales_data.head()

,Major,Minor,DocumentDate,SalePrice,ZipCode,SqFtTotLiving
0,4000,228,04/29/1997,103500,98168,1560
1,4000,228,10/15/2014,221900,98168,1560
2,4000,228,08/28/2020,0,98168,1560
3,4000,228,05/06/2005,198000,98168,1560
4,4000,228,04/26/2019,369000,98168,1560


In [44]:
sales_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1544897 entries, 0 to 1544896
Data columns (total 6 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   Major          1544897 non-null  object
 1   Minor          1544897 non-null  object
 2   DocumentDate   1544897 non-null  object
 3   SalePrice      1544897 non-null  int64 
 4   ZipCode        1404584 non-null  object
 5   SqFtTotLiving  1544897 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 82.5+ MB


We can see right away that we're missing ZIP codes for many of the sales transactions.

In [45]:
sales_data.loc[sales_data['ZipCode'].isna()].head()

,Major,Minor,DocumentDate,SalePrice,ZipCode,SqFtTotLiving
28,226700,160,05/08/2003,0,NaN,1560
29,226700,160,05/11/1996,0,NaN,1560
30,226700,160,09/08/2011,0,NaN,1560
31,226700,160,10/09/2018,855000,NaN,1560
32,226700,160,01/24/2020,0,NaN,1560


### Exercise

What percentage of housing records are missing ZIP codes?

In [46]:
sales_data['ZipCode'].isna().sum() / len(sales_data)

0.09082353063019735

<details>
    <summary>Answer</summary>
    <code>sales_data['ZipCode'].isna().sum() / sales_data.shape[0]</code>
    </details>

Let's drop the rows with missing zip codes.

In [47]:
sales_data = sales_data.dropna(subset=['ZipCode'], how='all')
sales_data.head()

,Major,Minor,DocumentDate,SalePrice,ZipCode,SqFtTotLiving
0,4000,228,04/29/1997,103500,98168,1560
1,4000,228,10/15/2014,221900,98168,1560
2,4000,228,08/28/2020,0,98168,1560
3,4000,228,05/06/2005,198000,98168,1560
4,4000,228,04/26/2019,369000,98168,1560


In [48]:
sales_data.isna().sum()

Major            0
Minor            0
DocumentDate     0
SalePrice        0
ZipCode          0
SqFtTotLiving    0
dtype: int64

In [49]:
sales_data = sales_data.loc[~sales_data['ZipCode'].isna(), :]
sales_data.head()

,Major,Minor,DocumentDate,SalePrice,ZipCode,SqFtTotLiving
0,4000,228,04/29/1997,103500,98168,1560
1,4000,228,10/15/2014,221900,98168,1560
2,4000,228,08/28/2020,0,98168,1560
3,4000,228,05/06/2005,198000,98168,1560
4,4000,228,04/26/2019,369000,98168,1560


## Time Permitting: Data Cleaning with Pandas

### 1. Investigate and drop rows with invalid values in the SalePrice and SqFtTotLiving columns.

In [50]:
sales_data.describe()

,SalePrice,SqFtTotLiving
count,1.404584e+06,1.404584e+06
mean,3.089713e+05,2.109228e+03
std,7.589833e+05,9.776886e+02
min,-4.000000e+02,0.000000e+00
25%,0.000000e+00,1.440000e+03
50%,1.675000e+05,1.940000e+03
75%,3.800000e+05,2.570000e+03
max,3.750000e+07,4.816000e+04


In [51]:
sales_data['SalePrice'].min()

-400

In [52]:
sales_data = sales_data[sales_data['SalePrice'] > 10000]

<details>
    <summary>One possible answer here</summary>
    <code>sales_data = sales_data[sales_data['SalePrice'] > 10000]</code>
    </details>

### 2. Investigate and handle non-numeric ZipCode values

Can you find a way to shorten ZIP+4 codes to the first five digits?

In [53]:
sales_data['ZipCode'].dtype

dtype('O')

In [56]:
def is_integer(x):
    try:
        _ = int(x)
    except ValueError:
        return False
    return True

sales_data.loc[sales_data['ZipCode'].apply(is_integer) == False, 'ZipCode'].head()

12206    98199-3014
12207    98199-3014
12208    98199-3014
20997    98028-8908
35256    98042-3001
Name: ZipCode, dtype: object

In [57]:
sales_data['ZipCode'].dtype

dtype('O')

In [58]:
sales_data['ZipCode'].sample(10)

720048     98103.0
1202870      98033
267681       98052
1419191      98115
556120     98188.0
788844       98126
1231181      98166
802055     98038.0
827853       98115
108278       98038
Name: ZipCode, dtype: object

In [59]:
def five_digit_ZIP(x):
    try:
        return int(str(x)[:5])
    except:
        return x
sales_data['ZipCode'] = sales_data['ZipCode'].map(five_digit_ZIP)
sales_data = sales_data.loc[sales_data['ZipCode'].apply(is_integer) == True, :]
sales_data['ZipCode'] = sales_data['ZipCode'].map(int)

<details>
    <summary>One possible answer here</summary>
    <code>def five_digit_ZIP(x):
    try:
        return int(str(x)[:5])
    except:
        return x
sales_data['ZipCode'] = sales_data['ZipCode'].map(five_digit_ZIP)
sales_data = sales_data.loc[sales_data['ZipCode'].apply(is_integer) == True, :]
sales_data['ZipCode'] = sales_data['ZipCode'].map(int)</code>
    </details>

In [60]:
sales_data.head(2)

,Major,Minor,DocumentDate,SalePrice,ZipCode,SqFtTotLiving
0,4000,228,04/29/1997,103500,98168,1560
1,4000,228,10/15/2014,221900,98168,1560


### 3. Add a column for PricePerSqFt



In [61]:
sales_data['PricePerSqFt'] =  sales_data['SalePrice']/sales_data['SqFtTotLiving']
sales_data['PricePerSqFt'].head()

0     66.346154
1    142.243590
3    126.923077
4    236.538462
7    217.415730
Name: PricePerSqFt, dtype: float64

<details>
    <summary>Answer here</summary>
    <code>sales_data['PricePerSqFt'] = sales_data['SalePrice'] / sales_data['SqFtTotLiving']</code>
    </details>

### 4. Subset the data to 2021 sales only.

We can assume that the DocumentDate is approximately the sale date.

In [62]:
sales_data ['DocumentDate'].dtype

dtype('O')

In [63]:
sales_data['DocumentDate'] = pd.to_datetime(sales_data['DocumentDate'])

In [64]:
sales_data ['DocumentDate'].sample(2)

1029612   2019-07-18
1066894   2005-08-23
Name: DocumentDate, dtype: datetime64[ns]

sales_data = sales_data[sales_data['DocumentDate'] >= '2021-01-01']
sales_data = sales_data[sales_data['DocumentDate'] < '2022-01-01']
sales_data.head()

In [66]:
sales_data = sales_data[(sales_data['DocumentDate'] >= '2021-01-01') & (sales_data['DocumentDate'] < '2022-01-01')]
sales_data.head()

,Major,Minor,DocumentDate,SalePrice,ZipCode,SqFtTotLiving,PricePerSqFt
38,891050,230,2021-12-13,920000,98133,2770,332.129964
46,118000,275,2021-09-29,311000,98178,2880,107.986111
147,923890,1045,2021-12-17,726000,98136,1230,590.243902
174,251701,640,2021-03-22,605000,98042,2290,264.192140
222,329370,160,2021-03-24,815000,98133,2500,326.000000


<details>
    <summary>Answer here</summary>
    <code>sales_data['DocumentDate'] = pd.to_datetime(sales_data['DocumentDate'])
sales_data = sales_data.loc[sales_data['DocumentDate'] > '12/31/2020']</code>
    </details>

### 5. What is the mean price per square foot for a house sold in Seattle in 2021?

In [67]:
sales_data['PricePerSqFt'].mean()

513.5327602840078

<details>
    <summary>Answer here</summary>
    <code>sales_data['PricePerSqFt'].mean()</code>
    </details>

## Level Up: `pandas.set_option()`

We can adjust how `pandas` works by setting options in advance.

### Block Scientific Notation

For example, suppose we want to prevent numbers from being displayed in scientific notation.

In [68]:
df = pd.DataFrame([[1e9, 2e9], [3e9, 4e9]])
df

,0,1
0,1.000000e+09,2.000000e+09
1,3.000000e+09,4.000000e+09


Then we can use:

In [69]:
pd.set_option('display.float_format', '{:.2f}'.format)
df

,0,1
0,1000000000.00,2000000000.00
1,3000000000.00,4000000000.00


### See More Rows

Or suppose we want `pandas` to show more rows.

In [70]:
df2 = pd.DataFrame(np.array(range(100)))
df2

,0
0,0
1,1
2,2
3,3
4,4
...,...
95,95
96,96
97,97
98,98


In that case we can use:

In [71]:
pd.set_option('display.max_rows', 100)

df2

,0
0,0
1,1
2,2
3,3
4,4
5,5
6,6
7,7
8,8
9,9


For complete documentation, see [here](https://pandas.pydata.org/pandas-docs/stable/user_guide/options.html).